In [3]:
import pandas as pd
import numpy as np, re, time, pickle, os

df = pd.read_parquet("prices_feat.parquet") 

In [4]:
from openai import OpenAI, APITimeoutError, RateLimitError

os.environ["OPENAI_API_KEY"] = "OPENAI_API_KEY"

client = OpenAI()
print("client OK")

client OK


In [10]:
FT_MODEL  = "ft:gpt-3.5-turbo-0125:mingjun-wu:stock-return-v1:Bcj9Js22"

In [6]:
feat_cols = ["momentum_rsi", "trend_macd", "trend_sma_fast", "volatility_atr",
    "momentum_stoch_rsi", "trend_adx", "trend_cci", "volume_obv",
    "volatility_bbh", "volume_cmf"]
LOOKBACK  = 30

In [7]:
def row_to_line(date_str, row):
    return date_str + " " + ", ".join(f"{k}:{row[k]:.2g}" for k in feat_cols)

_float_re = re.compile(r"[-+]?\d*\.?\d+(?:[eE][-+]?\d+)?")
def parse_float(txt, default=0.0):
    m = _float_re.search(txt)
    return float(m.group()) if m else default

def daily_sharpe(signal, returns, dates):
    df_p = pd.DataFrame({"Date": dates, "sig": signal, "ret": returns})
    daily = (df_p.groupby("Date", group_keys=False)
                   .apply(lambda g: g.loc[g.sig==1,"ret"].mean()
                                - g.loc[g.sig==-1,"ret"].mean())
                   .dropna())
    return daily.mean() / daily.std(ddof=1) * np.sqrt(252)

In [12]:
TEST_DF   = df[df["Date"].dt.year == 2024].reset_index(drop=True)
CACHE     = "ft_preds.pkl"
preds_ft  = pickle.load(open(CACHE,"rb")) if os.path.exists(CACHE) else []
done_idx  = {i for i,_ in preds_ft}

for i in range(LOOKBACK, len(TEST_DF)):
    if i in done_idx:          
        continue
    cur  = TEST_DF.iloc[i]
    hist = TEST_DF.iloc[i-LOOKBACK:i]
    if (hist["Ticker"] != cur["Ticker"]).any():
        continue                

    prompt = (
        f"You are a quantitative analyst.\n"
        f"Below are {LOOKBACK} consecutive trading days of technical indicators for {cur['Ticker']}.\n"
        "Predict the percentage return for the NEXT trading day. Respond with a single decimal number.\n\n" +
        "\n".join(row_to_line(d.strftime('%Y-%m-%d'), r)
                  for d,r in zip(hist["Date"], hist.to_dict('records')))
    )

    try:
        resp = client.chat.completions.create(
            model      = FT_MODEL,
            messages   = [{"role":"user","content":prompt}],
            temperature= 0,
            timeout    = 30
        )
        y_hat = parse_float(resp.choices[0].message.content)
    except (RateLimitError, APITimeoutError) as e:
        print("API error:", e)
        time.sleep(10)
        continue

    preds_ft.append((i, y_hat))
    if len(preds_ft) % 50 == 0:
        pickle.dump(preds_ft, open(CACHE,"wb"))
        print(f"finished {len(preds_ft)} / {len(TEST_DF)-LOOKBACK}")
    time.sleep(1.3)  

pickle.dump(preds_ft, open(CACHE,"wb"))
print("finished, #samples：", len(preds_ft))

finished 1100 / 1220
finished, #samples： 1100


In [13]:
idx, y_hat = zip(*preds_ft)
y_pred = np.array(y_hat)
y_true = TEST_DF.loc[list(idx), "return_fwd"].values
dates  = TEST_DF.loc[list(idx), "Date"]

mse  = np.mean((y_true - y_pred)**2)
rmse = np.sqrt(mse)
sig  = np.sign(y_pred); sig[sig==0] = -1
sharpe = daily_sharpe(sig, y_true, dates)

print("\n=== Fine‑tuned LLM • Test‑2024 ===")
print({"MSE": round(mse,6),
       "RMSE": round(rmse,6),
       "Sharpe": round(sharpe,3)})


=== Fine‑tuned LLM • Test‑2024 ===
{'MSE': 0.000318, 'RMSE': 0.017839, 'Sharpe': -19.661}


C:\Users\eddie\AppData\Local\Temp\ipykernel_75488\4017433017.py:11: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  daily = (df_p.groupby("Date", group_keys=False)
